# Working with the EIA Extract / Transform
This notebook steps through PUDL's extract and transform steps for the EIA 860 and 923 datasets, to make it easier to test and add new years of data, or new tables from the various spreadsheets that haven't been integrated yet.

In [1]:
%load_ext autoreload
%autoreload 3
import pudl
import logging
import sys
from pathlib import Path
import pandas as pd
pd.options.display.max_columns = None

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [3]:
pudl_settings = pudl.workspace.setup.get_defaults()

## Set the scope for the Extract-Transform:

In [4]:
from pudl.settings import Eia860Settings, Eia923Settings, EiaSettings
from pudl.metadata.classes import DataSource

eia860_data_source = DataSource.from_id("eia860")
eia860_settings = Eia860Settings(
# Limit the years as needed if you're testing only a few of them. E.g.:
#    years=[2021],
    years=eia860_data_source.working_partitions["years"]
# By default all of the tables will be processed.
# Select the relevant tables as needed if you're testing only a few of them. E.g.:
#   tables=["generation_fuel_nuclear_eia923", "generation_fuel_eia923"]
)

eia923_data_source = DataSource.from_id("eia923")
eia923_settings = Eia923Settings(
# Limit the years as needed if you're testing only a few of them. E.g.:
#    years = [2021]
    years = eia923_data_source.working_partitions["years"]
# By default all of the tables will be processed.
# Select the relevant tables as needed if you're testing only a few of them. E.g.:
#   tables=["generation_fuel_nuclear_eia923", "generation_fuel_eia923"]
)

eia_settings = EiaSettings(eia860=eia860_settings, eia923=eia923_settings)

## Create a locally cached datastore

In [5]:
ds_kwargs = {"local_cache_path": pudl_settings["data_dir"]}
ds = pudl.workspace.datastore.Datastore(**ds_kwargs)

# EIA-860

## Extract just the EIA-860 / EIA-860m

In [6]:
%%time
eia860_extractor = pudl.extract.eia860.Extractor(ds)
eia860_raw_dfs = eia860_extractor.extract(settings=eia860_settings)

eia860m_extractor = pudl.extract.eia860m.Extractor(ds)
if eia860_settings.eia860m:
    eia860m_raw_dfs = eia860m_extractor.extract(
        settings=eia860_settings
    )
    eia860_raw_dfs = pudl.extract.eia860m.append_eia860m(
        eia860_raw_dfs=eia860_raw_dfs,
        eia860m_raw_dfs=eia860m_raw_dfs
    )

2023-03-20 10:09:03 [    INFO] catalystcoop.pudl.extract.excel:237 Extracting eia860 spreadsheet data.


Extracting eia860 spreadsheet data.
boiler_cool
boiler_fgd
boiler_fgp
boiler_gen
boiler_stackflue
Boiler Cooler
Boiler FGD
Boiler FGP
Boiler Generator
Boiler Stack Flue
Boiler Cooling
Boiler Generator
Boiler Mercury
Boiler NOx
Boiler Particulate Matter
Boiler SO2
Boiler Stack Flue
Emissions Control Equipment
Boiler Cooling
Boiler Generator
Boiler Mercury
Boiler NOx
Boiler Particulate Matter
Boiler SO2
Boiler Stack Flue
Emissions Control Equipment
Boiler Cooling
Boiler Generator
Boiler Mercury
Boiler NOx
Boiler Particulate Matter
Boiler SO2
Boiler Stack Flue
Emissions Control Equipment
Boiler Cooling
Boiler Generator
Boiler Mercury
Boiler NOx
Boiler Particulate Matter
Boiler SO2
Boiler Stack Flue
Emissions Control Equipment
Emissions Control Equipment
Boiler Cooling
Boiler Generator
Boiler Mercury
Boiler NOx
Boiler Particulate Matter
Boiler SO2
Boiler Stack Flue
Emissions Control Equipment
Boiler Cooling
Boiler Generator
Boiler Mercury
Boiler NOx
Boiler Particulate Matter
Boiler SO2
Boi

/home/knordback/Kurt/pudl/src/pudl/helpers.py:1556: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/home/knordback/Kurt/pudl/src/pudl/helpers.py:1556: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/home/knordback/Kurt/pudl/src/pudl/helpers.py:1556: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


operable
proposed
retired & canceled
Operable
Proposed
Retired and Canceled
Operable
Proposed
Retired and Canceled
Operable
Proposed
Retired and Canceled
Operable
Proposed
Retired and Canceled
Operable
Proposed
Retired and Canceled
Operable
Proposed
Retired and Canceled
Operable
Proposed
Retired and Canceled
Operable
Proposed
Retired and Canceled
Operable
Proposed
Retired and Canceled
Operable
Proposed
Retired and Canceled


/home/knordback/Kurt/pudl/src/pudl/helpers.py:1556: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/home/knordback/Kurt/pudl/src/pudl/helpers.py:1556: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/home/knordback/Kurt/pudl/src/pudl/helpers.py:1556: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


...

plant2011
plant2012
Plant
Plant
Plant
Plant
Plant
Plant
Plant
Plant
Plant


/home/knordback/Kurt/pudl/src/pudl/helpers.py:1556: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/home/knordback/Kurt/pudl/src/pudl/helpers.py:1556: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/home/knordback/Kurt/pudl/src/pudl/helpers.py:1556: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


utility2011
utility2012
Utility
Utility
Utility
Utility
Utility
Utility
Utility
Utility
Utility


2023-03-20 10:18:27 [    INFO] catalystcoop.pudl.extract.excel:237 Extracting eia860m spreadsheet data.


Extracting eia860m spreadsheet data.
Operating
Canceled or Postponed
Operating_PR
Planned
Planned_PR
Retired
Retired_PR
CPU times: user 9min 40s, sys: 1.52 s, total: 9min 42s
Wall time: 9min 42s


## Transform EIA-860 / EIA-860m

In [7]:
%%time
eia860_transformed_dfs = pudl.transform.eia860.transform(
    eia860_raw_dfs,
    eia860_settings=eia860_settings,
)

2023-03-20 10:18:46 [    INFO] catalystcoop.pudl.transform.eia860:817 Transforming raw EIA 860 DataFrames for ownership_eia860 concatenated across all years.


Transforming raw EIA 860 DataFrames for ownership_eia860 concatenated across all years.


2023-03-20 10:19:01 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding ownership_eia860.data_maturity


Recoding ownership_eia860.data_maturity


2023-03-20 10:19:01 [    INFO] catalystcoop.pudl.transform.eia860:817 Transforming raw EIA 860 DataFrames for generators_eia860 concatenated across all years.


...

Recoding boiler_generator_assn_eia860.steam_plant_type_code


2023-03-20 10:21:58 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding boiler_generator_assn_eia860.data_maturity


Recoding boiler_generator_assn_eia860.data_maturity


2023-03-20 10:21:58 [    INFO] catalystcoop.pudl.transform.eia860:817 Transforming raw EIA 860 DataFrames for utilities_eia860 concatenated across all years.


Transforming raw EIA 860 DataFrames for utilities_eia860 concatenated across all years.
CPU times: user 2min 55s, sys: 20.2 s, total: 3min 15s
Wall time: 3min 15s


# EIA-923

## Extract just the EIA-923

In [8]:
%%time
eia923_extractor = pudl.extract.eia923.Extractor(ds)
eia923_raw_dfs = eia923_extractor.extract(settings=eia_settings.eia923)

2023-03-20 10:22:02 [    INFO] catalystcoop.pudl.extract.excel:237 Extracting eia923 spreadsheet data.


Extracting eia923 spreadsheet data.
Page 2 Oil Stocks Data
Page 1 Generation and Fuel Data
Page 2 Coal Stocks Data
Page 2 Petcoke Stocks Data
Page 2 Stocks Data
Page 3 Boiler Fuel Data
Page 4 Generator Data
Page 5 Fuel Receipts and Costs
Page 6 Plant Frame
Page 2 Oil Stocks Data
Page 1 Generation and Fuel Data
Page 2 Coal Stocks Data
Page 2 Petcoke Stocks Data
Page 2 Stocks Data
Page 3 Boiler Fuel Data
Page 4 Generator Data
Page 5 Fuel Receipts and Costs
Page 6 Plant Frame
Page 2 Oil Stocks Data
Page 1 Generation and Fuel Data
Page 2 Coal Stocks Data
Page 2 Petcoke Stocks Data
Page 2 Stocks Data
Page 3 Boiler Fuel Data
Page 4 Generator Data
Page 5 Fuel Receipts and Costs
Page 6 Plant Frame
Page 1 Generation and Fuel Data
Page 1 Energy Storage
Page 2 Coal Stocks Data
Page 2 Oil Stocks Data
Page 2 Petcoke Stocks Data
Page 2 Stocks Data
Page 3 Boiler Fuel Data
Page 4 Generator Data
Page 5 Fuel Receipts and Costs
Page 6 Plant Frame
Page 1 Generation and Fuel Data
Page 1 Energy Storage
Page

## Transform just the EIA-923

In [9]:
%%time
eia923_transformed_dfs = pudl.transform.eia923.transform(
    eia923_raw_dfs,
    eia923_settings=eia923_settings,
)

2023-03-20 10:30:26 [    INFO] catalystcoop.pudl.transform.eia923:1216 Transforming raw EIA 923 DataFrames for generation_fuel_eia923 concatenated across all years.


Transforming raw EIA 923 DataFrames for generation_fuel_eia923 concatenated across all years.


2023-03-20 10:31:44 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding generation_fuel_eia923.energy_source_code


Recoding generation_fuel_eia923.energy_source_code


2023-03-20 10:31:44 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding generation_fuel_eia923.fuel_type_code_aer


...

Recoding fuel_receipts_costs_eia923.primary_transportation_mode_code


2023-03-20 10:33:34 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding fuel_receipts_costs_eia923.secondary_transportation_mode_code


Recoding fuel_receipts_costs_eia923.secondary_transportation_mode_code


2023-03-20 10:33:34 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding fuel_receipts_costs_eia923.data_maturity


Recoding fuel_receipts_costs_eia923.data_maturity
CPU times: user 3min 1s, sys: 5.79 s, total: 3min 7s
Wall time: 3min 7s


# Combined EIA Data

## Merge EIA-923/860, set dtypes, harvest entities

In [10]:
%%time
eia_transformed_dfs = eia923_transformed_dfs.copy()
eia_transformed_dfs.update(eia860_transformed_dfs.copy())

# Do some final cleanup and assign appropriate types:
eia_transformed_dfs = {
    name: pudl.helpers.convert_cols_dtypes(df, data_source="eia")
    for name, df in eia_transformed_dfs.items()
}
    
entities_dfs, eia_transformed_dfs = pudl.transform.eia.transform(
    eia_transformed_dfs,
    eia_settings=eia_settings,
)

# Assign appropriate types to new entity tables:
entities_dfs = {
    name: pudl.metadata.fields.apply_pudl_dtypes(df, group="eia")
    for name, df in entities_dfs.items()
}

for table in entities_dfs:
    entities_dfs[table] = (
        pudl.metadata.classes.Package.from_resource_ids()
        .get_resource(table)
        .encode(entities_dfs[table])
    )

out_dfs = pudl.etl._read_static_encoding_tables(etl_group="static_eia")
out_dfs.update(entities_dfs)
out_dfs.update(eia_transformed_dfs)

2023-03-20 10:34:43 [    INFO] catalystcoop.pudl.transform.eia:1180 Harvesting IDs & consistently static attributes for EIA plants


Harvesting IDs & consistently static attributes for EIA plants


2023-03-20 10:35:02 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding plants_eia860.balancing_authority_code_eia


Recoding plants_eia860.balancing_authority_code_eia


2023-03-20 10:35:11 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding plants_eia860.reporting_frequency_code


...

Spot fixing incorrect PACW/PACE BA codes and names.


2023-03-20 10:43:13 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding boilers_entity_eia.boiler_manufacturer_code


Recoding boilers_entity_eia.boiler_manufacturer_code


2023-03-20 10:43:13 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding boilers_entity_eia.prime_mover_code


Recoding boilers_entity_eia.prime_mover_code
CPU times: user 9min 27s, sys: 9.99 s, total: 9min 37s
Wall time: 9min 38s
